In [2]:
import pandas as pd
from sqlalchemy import create_engine
from decouple import config

USER_DB = config("USER_DB", default='postgres')
PASSWORD_DB = config("PASSWORD_DB", default='password')
URL_DB = config("URL_DB", default='localhost')
NAME_DB = config("NAME_DB", default='db_name')
PORT_DB = config("PORT_DB", default='5432')
DB_URL = f"postgresql://{USER_DB}:{PASSWORD_DB}@{URL_DB}:{PORT_DB}/{NAME_DB}"


engine = create_engine(DB_URL)

In [3]:
df_pib = pd.read_sql('SELECT * FROM "PIP_Cepea"', con=engine, index_col='index')

df_pib.head()

,ano,insumos,agropecuária,industria,servicos,total,ramo,pib_renda
index,,,,,,,,
0,1996,31597.663029,200211.777672,514755.882623,646536.982537,1.393102e+06,agronegocio,True
1,1997,32825.998197,199911.301734,482894.842674,604714.559307,1.320347e+06,agronegocio,True
2,1998,31818.410647,200969.741431,455966.002147,576123.413175,1.264878e+06,agronegocio,True
3,1999,34784.067160,206952.587184,453199.228768,573119.731296,1.268056e+06,agronegocio,True
4,2000,40682.954859,209175.390473,496957.470676,626206.339831,1.373022e+06,agronegocio,True


In [4]:
df_prod = pd.read_sql('SELECT * FROM "producao_2019"', con=engine, index_col='index')

df_prod.head()

,area_abril,area_maio,area_junho,prod_abril,prod_maio,prod_junho,uf
index,,,,,,,
0,45842,80283,80385,95917,105458,105525,ALAGOAS
1,21548,22442,23242,50813,60375,62955,AMAPÁ
2,22930,22930,22930,41207,41420,41420,AMAZONAS
3,3130065,3107865,3085865,8283660,9065031,9359331,BAHIA
4,909518,961926,961640,559791,918483,903649,CEARÁ


### Qual a porcentagem de participação da região Centro-Oeste na produção nacional no primeiro semestre de 2019?

In [5]:
total_br = df_prod[
    ['prod_abril', 'prod_maio', 'prod_junho']
].sum().sum()

f'Produção nacional: {total_br}t'

'Total nacional em toneladas: 734492892'

In [18]:
total_co = df_prod[
    df_prod['uf'].isin(
        ['DISTRITO FEDERAL', 'GOIÁS', 'MATO GROSSO', 'MATO GROSSO DO SUL']
    )
][
    ['prod_abril', 'prod_maio', 'prod_junho']
].sum().sum()

f'Produção da região centro-oeste: {total_co}t'

'Produção da região centro-oeste: 343321217t'

In [7]:
co_percent = total_co / (total_br / 100)

f'{total_co} = {round(co_percent, 2)}% de {total_br}'

'343321217 = 46.74% de 734492892'

### Qual estado possui maior área produtiva? 

In [8]:
df_prod[
    df_prod['area_junho'] == df_prod['area_junho'].max()
]['uf']

index
9    MATO GROSSO
Name: uf, dtype: object

### Qual a média de produção mensal para o estado de São Paulo?

In [19]:
df_prod[df_prod['uf'] == 'SÃO PAULO'][
    ['prod_abril', 'prod_maio', 'prod_junho']
].sum().mean()

9604077.0

### Defina a correlação entre a produção agrícola e o PIB do Ramo Agrícola e Pecuário em 2019. Qual segmento possui maior correlação e qual possui menor correlação? Demonstre num gráfico e explique os coeficientes.